In [ ]:
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt
from scipy.fftpack import fftn, fftshift
from skimage.filters import sobel
from skimage.feature import graycomatrix, graycoprops
from skimage.color import rgb2gray
from skimage import feature, filters, measure

# Función para cargar una imagen en escala de grises
def load_image(image_path):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        raise ValueError("No se pudo cargar la imagen")
    return img


# Función para combinar múltiples imágenes 2D en una imagen 3D
def stack_images_to_3d(folder_path):
    image_files = sorted([f for f in os.listdir(folder_path) if f.endswith(('.png', '.jpg', '.tif'))])
    if not image_files:
        raise ValueError("No se encontraron imágenes en la carpeta")
    
    images = [load_image(os.path.join(folder_path, f)) for f in image_files]
    volume = np.stack(images, axis=-1)  # Apila las imágenes en la tercera dimensión
    return volume

# Función para calcular la Transformada de Fourier 3D de la imagen
def compute_fft_3d(img):
    f_transform = fftn(img)  # Calcula la transformada de Fourier en 3D
    f_transform_shifted = fftshift(f_transform)  # Centra la transformada de Fourier
    magnitude_spectrum = np.log1p(np.abs(f_transform_shifted))  # Escala logarítmica para visualización
    return magnitude_spectrum

# Función para calcular el tensor de estructura, que mide la variación direccional de la imagen en 2D
def compute_structure_tensor2D(img):
    gradient_x = sobel(img, axis=0)  # Derivada en la dirección X
    gradient_y = sobel(img, axis=1)  # Derivada en la dirección Y

    # Cálculo de los elementos del tensor de estructura
    Jxx = gradient_x * gradient_x
    Jyy = gradient_y * gradient_y
    Jxy = gradient_x * gradient_y

    # Determinante de la matriz de orientación (sin considerar el eje Z)
    DoI = (Jxx + Jyy) - np.sqrt((Jxx - Jyy)**2 + 4*(Jxy**2))
    return DoI




# Función para calcular el tensor de estructura, que mide la variación direccional de la imagen
def compute_structure_tensor(img):
    gradient_x = sobel(img, axis=0)  # Derivada en la dirección X
    gradient_y = sobel(img, axis=1)  # Derivada en la dirección Y
    gradient_z = sobel(img, axis=2)  # Derivada en la dirección Z
    
    # Cálculo de los elementos del tensor de estructura
    Jxx = gradient_x * gradient_x
    Jyy = gradient_y * gradient_y
    Jzz = gradient_z * gradient_z
    Jxy = gradient_x * gradient_y
    Jxz = gradient_x * gradient_z
    Jyz = gradient_y * gradient_z
    
    # Determinante de la matriz de orientación
    DoI = (Jxx + Jyy + Jzz) - np.sqrt((Jxx - Jyy)**2 + (Jyy - Jzz)**2 + (Jzz - Jxx)**2 + 4*(Jxy**2 + Jxz**2 + Jyz**2))
    return DoI

# Función para detectar bordes en imágenes 3D usando el algoritmo de Canny
def detect_edges_3d(img):
    return feature.canny(img, sigma=1)  # Aplicación de la detección de bordes con sigma=1

# Función para analizar la textura de la imagen utilizando la matriz de co-ocurrencia de niveles de gris (GLCM)
def texture_analysis(img):
    img_gray = (img / img.max() * 255).astype(np.uint8)  # Normaliza la imagen a valores de 0-255
    glcm = graycomatrix(img_gray, distances=[1], angles=[0, np.pi/4, np.pi/2, 3*np.pi/4], levels=256, symmetric=True, normed=True)  # Calcula la GLCM
    contrast = graycoprops(glcm, 'contrast')  # Calcula el contraste de textura
    energy = graycoprops(glcm, 'energy')  # Calcula la energía de textura
    return contrast.mean(), energy.mean()

# Función principal para ejecutar el análisis de la imagen
def main2D(image_path):
    img = load_image(image_path)  # Cargar la imagen
    fft_result = compute_fft_3d(img)  # Calcular la transformada de Fourier 3D
    structure_tensor = compute_structure_tensor2D(img)  # Calcular el tensor de estructura
    edges = detect_edges_3d(img)  # Detectar bordes con Canny en 3D
    contrast, energy = texture_analysis(img)  # Análisis de textura
    
    # Imprimir los resultados del análisis de textura
    print(f"Contraste de textura: {contrast:.3f}, Energía de textura: {energy:.3f}")
    
    # Visualización de los resultados
    plt.figure(figsize=(12, 4))
    plt.subplot(1, 4, 1)
    plt.imshow(img, cmap='gray')
    plt.title('Imagen Original')
    plt.axis('off')
    
    plt.subplot(1, 4, 2)
    plt.imshow(fft_result, cmap='inferno')
    plt.title('Transformada de Fourier 3D')
    plt.axis('off')
    
    plt.subplot(1, 4, 3)
    plt.imshow(structure_tensor, cmap='viridis')
    plt.title('Tensor de Estructura')
    plt.axis('off')
    
    plt.subplot(1, 4, 4)
    plt.imshow(edges, cmap='gray')
    plt.title('Detección de Bordes 3D')
    plt.axis('off')
    
    plt.show()


# Función principal para ejecutar el análisis de todas las imágenes en una carpeta
def main2D_folder(folder_path):
    image_files = sorted([f for f in os.listdir(folder_path) if f.endswith(('.png', '.jpg', '.tif'))])
    if not image_files:
        raise ValueError("No se encontraron imágenes en la carpeta")
    
    for image_file in image_files:
        image_path = os.path.join(folder_path, image_file)
        img = load_image(image_path)  # Cargar la imagen
        fft_result = compute_fft_3d(img)  # Calcular la transformada de Fourier 3D
        structure_tensor = compute_structure_tensor2D(img)  # Calcular el tensor de estructura
        edges = detect_edges_3d(img)  # Detectar bordes con Canny en 3D
        contrast, energy = texture_analysis(img)  # Análisis de textura
        
        # Imprimir los resultados del análisis de textura
        print(f"Imagen: {image_file}")
        print(f"Contraste de textura: {contrast:.3f}, Energía de textura: {energy:.3f}")
        
        # Visualización de los resultados
        plt.figure(figsize=(12, 4))
        plt.subplot(1, 4, 1)
        plt.imshow(img, cmap='gray')
        plt.title('Imagen Original')
        plt.axis('off')
        
        plt.subplot(1, 4, 2)
        plt.imshow(fft_result, cmap='inferno')
        plt.title('Transformada de Fourier 3D')
        plt.axis('off')
        
        plt.subplot(1, 4, 3)
        plt.imshow(structure_tensor, cmap='viridis')
        plt.title('Tensor de Estructura')
        plt.axis('off')
        
        plt.subplot(1, 4, 4)
        plt.imshow(edges, cmap='gray')
        plt.title('Detección de Bordes 3D')
        plt.axis('off')
        
        plt.show()




In [ ]:
main2D("C:\\Users\\danwo\\Desktop\\ZS0001_TI_25XW_Rhodopsin_Au\\IVM4101#_TI_25XW_Rhodopsin_Au.png")


In [ ]:
main2D_folder("C:\\Users\\danwo\\Desktop\\ZS0001_TI_25XW_Rhodopsin_Au")

In [ ]:
# Función para calcular la Transformada de Fourier 2D de la imagen
def compute_fft_2d(img):
    f_transform = fftn(img)  # Calcula la transformada de Fourier en 2D
    f_transform_shifted = fftshift(f_transform)  # Centra la transformada de Fourier
    magnitude_spectrum = np.log1p(np.abs(f_transform_shifted))  # Escala logarítmica para visualización
    return f_transform_shifted, magnitude_spectrum

# Función para detectar las direcciones principales en el espectro de Fourier
def detect_main_directions(magnitude_spectrum):
    # Encuentra los índices de los picos en el espectro de magnitud
    indices = np.unravel_index(np.argsort(magnitude_spectrum.ravel())[::-1], magnitude_spectrum.shape)
    main_directions = list(zip(indices[0][:2], indices[1][:2]))
    return main_directions

# Función para aplicar un filtro que mantenga solo las frecuencias en las direcciones principales
def filter_main_directions(f_transform, main_directions, radius=5):
    # Crear una máscara vacía del mismo tamaño que la transformada
    mask = np.zeros_like(f_transform, dtype=bool)
    
    # Rellenar la máscara alrededor de las direcciones principales
    for direction in main_directions:
        y, x = direction
        # Crear una región alrededor de la dirección principal
        y_min, y_max = max(0, y - radius), min(f_transform.shape[0], y + radius + 1)
        x_min, x_max = max(0, x - radius), min(f_transform.shape[1], x + radius + 1)
        
        # Aplica la máscara en el rango definido
        mask[y_min:y_max, x_min:x_max] = True
    
    # Aplicar la máscara al espectro de Fourier
    filtered_transform = np.zeros_like(f_transform)
    filtered_transform[mask] = f_transform[mask]
    
    return filtered_transform



# Función principal para ejecutar el análisis de todas las imágenes en una carpeta
def main2D_folder_with_filter(folder_path):
    image_files = sorted([f for f in os.listdir(folder_path) if f.endswith(('.png', '.jpg', '.tif'))])
    if not image_files:
        raise ValueError("No se encontraron imágenes en la carpeta")
    
    for image_file in image_files:
        image_path = os.path.join(folder_path, image_file)
        img = load_image(image_path)  # Cargar la imagen
        f_transform_shifted, magnitude_spectrum = compute_fft_2d(img)  # Calcular la transformada de Fourier 2D
        
        main_directions = detect_main_directions(magnitude_spectrum)  # Detectar las direcciones principales
        filtered_transform = filter_main_directions(f_transform_shifted, main_directions)  # Filtrar las direcciones principales
        
        # Calcular el espectro de magnitud del espectro filtrado
        filtered_magnitude_spectrum = np.log1p(np.abs(filtered_transform))
        
        # Visualización de los resultados
        plt.figure(figsize=(12, 4))
        plt.subplot(1, 3, 1)
        plt.imshow(img, cmap='gray')
        plt.title('Imagen Original')
        plt.axis('off')
        
        plt.subplot(1, 3, 2)
        plt.imshow(magnitude_spectrum, cmap='inferno')
        plt.title('Transformada de Fourier 2D')
        plt.axis('off')
        
        plt.subplot(1, 3, 3)
        plt.imshow(filtered_magnitude_spectrum, cmap='inferno')
        plt.title('Espectro Filtrado')
        plt.axis('off')
        
        plt.show()


In [ ]:
main2D_folder_with_filter("C:\\Users\\danwo\\Desktop\\ZS0001_TI_25XW_Rhodopsin_Au")


Pasa que quedarse con las 2 direcciones principales es muy restrictivo porque solo se queda con un punto es por ello que ahora voy a probar con un filtro menos restrictivo para eliminar simplemente el ruido 

In [ ]:
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt
from scipy.fftpack import fftn, fftshift, ifftn

# Función para cargar una imagen en escala de grises
def load_image(image_path):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        raise ValueError(f"No se pudo cargar la imagen: {image_path}")
    return img

# Función para calcular la Transformada de Fourier 2D
def compute_fft_2d(img):
    f_transform = fftn(img)  
    f_transform_shifted = fftshift(f_transform)  
    magnitude_spectrum = np.log1p(np.abs(f_transform_shifted))  
    return f_transform_shifted, magnitude_spectrum

# Función para filtrar los valores más altos de la Transformada de Fourier
def filter_high_values(f_transform, threshold_percent=90):
    magnitude = np.abs(f_transform)
    threshold = np.percentile(magnitude, threshold_percent)  # Calcula el umbral basado en percentiles
    mask = magnitude >= threshold  # Máscara booleana: True donde se conserva la información
    filtered_transform = f_transform * mask  # Aplica la máscara conservando solo valores altos
    print(f"Umbral aplicado: {threshold}")  # Depuración
    return filtered_transform

# Función principal para procesar todas las imágenes en una carpeta
def process_images_in_folder(folder_path, threshold_percent=10):
    image_files = sorted([f for f in os.listdir(folder_path) if f.endswith(('.png', '.jpg', '.tif'))])
    if not image_files:
        raise ValueError("No se encontraron imágenes en la carpeta")

    for image_file in image_files:
        image_path = os.path.join(folder_path, image_file)
        img = load_image(image_path)  # Cargar la imagen
        f_transform_shifted, magnitude_spectrum = compute_fft_2d(img)  # Transformada de Fourier 2D
        
        filtered_transform = filter_high_values(f_transform_shifted, threshold_percent)  # Filtrar espectro
        
        # Espectro filtrado para visualizar
        filtered_magnitude_spectrum = np.log1p(np.abs(filtered_transform))
        
        # Reconstrucción de la imagen desde la transformada filtrada
        reconstructed_img = np.abs(ifftn(filtered_transform))

        # Visualización de resultados
        plt.figure(figsize=(16, 4))
        plt.subplot(1, 4, 1)
        plt.imshow(img, cmap='gray')
        plt.title('Imagen Original')
        plt.axis('off')
        
        plt.subplot(1, 4, 2)
        plt.imshow(magnitude_spectrum, cmap='inferno')
        plt.title('Transformada de Fourier 2D')
        plt.axis('off')
        
        plt.subplot(1, 4, 3)
        plt.imshow(filtered_magnitude_spectrum, cmap='inferno')
        plt.title(f'Espectro Filtrado ({threshold_percent}%)')
        plt.axis('off')
        
        plt.subplot(1, 4, 4)
        plt.imshow(reconstructed_img, cmap='gray')
        plt.title('Imagen Reconstruida')
        plt.axis('off')
        
        plt.show()

# 👉 Usa esta línea para ejecutar la función con una carpeta específica
process_images_in_folder("C:\\Users\\danwo\\Desktop\\ZS0001_TI_25XW_Rhodopsin_Au", threshold_percent=90)


El anterior no se porque pero no me funciona porque se afectar el umbral por lo que algo está fallando. Ahora lo que voy a hacer es un filtro primero sencillo de anillos para ver que parte condensa la información y luego un filtro más avanzado para según el analisis anterior quedarme con la información relevante de forma más sotisficada


In [ ]:
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt
from scipy.fftpack import fftn, fftshift, ifftn

# Función para cargar una imagen en escala de grises
def load_image(image_path):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        raise ValueError(f"No se pudo cargar la imagen: {image_path}")
    return img

# Función para calcular la Transformada de Fourier 2D
def compute_fft_2d(img):
    f_transform = fftn(img)  
    f_transform_shifted = fftshift(f_transform)  
    magnitude_spectrum = np.log1p(np.abs(f_transform_shifted))  
    return f_transform_shifted, magnitude_spectrum

# Opción 1: Filtrar los valores de Fourier según porcentaje de energía
def filter_by_energy(f_transform, energy_percent=90):
    magnitude = np.abs(f_transform)
    total_energy = np.sum(magnitude)  # Energía total de la imagen en la frecuencia
    sorted_magnitude = np.sort(magnitude.ravel())[::-1]  # Ordenamos de mayor a menor
    cumulative_energy = np.cumsum(sorted_magnitude)  # Suma acumulativa de energía
    
    # Encontramos el umbral donde alcanzamos el porcentaje de energía deseado
    threshold_index = np.searchsorted(cumulative_energy, total_energy * (energy_percent / 100))
    threshold_value = sorted_magnitude[threshold_index]

    # Creamos la máscara conservando solo los valores con suficiente energía
    mask = magnitude >= threshold_value
    filtered_transform = f_transform * mask  # Aplicamos el filtro
    
    print(f"Umbral de energía aplicado: {threshold_value:.4f} (Conserva {energy_percent}% de la energía)")
    return filtered_transform

# Opción 2: Filtro de anillo para conservar frecuencias específicas
def filter_by_frequency_ring(f_transform, low_radius=20, high_radius=80):
    h, w = f_transform.shape
    cy, cx = h // 2, w // 2  # Centro de la imagen
    
    y, x = np.ogrid[:h, :w]  # Mallas de coordenadas
    distance = np.sqrt((x - cx)**2 + (y - cy)**2)  # Distancia radial al centro

    # Creamos la máscara que mantiene un anillo entre dos radios
    mask = (distance >= low_radius) & (distance <= high_radius)
    filtered_transform = f_transform * mask  # Aplicamos la máscara
    
    print(f"Filtro de anillo: Mantiene frecuencias entre {low_radius}-{high_radius} píxeles")
    return filtered_transform

# Función principal para procesar todas las imágenes en una carpeta
def process_images_in_folder(folder_path, energy_percent=90, use_ring_filter=False, low_radius=20, high_radius=80):
    image_files = sorted([f for f in os.listdir(folder_path) if f.endswith(('.png', '.jpg', '.tif'))])
    if not image_files:
        raise ValueError("No se encontraron imágenes en la carpeta")

    for image_file in image_files:
        image_path = os.path.join(folder_path, image_file)
        img = load_image(image_path)  # Cargar la imagen
        f_transform_shifted, magnitude_spectrum = compute_fft_2d(img)  # Transformada de Fourier 2D
        
        # Elegimos qué método de filtrado usar
        if use_ring_filter:
            filtered_transform = filter_by_frequency_ring(f_transform_shifted, low_radius, high_radius)
        else:
            filtered_transform = filter_by_energy(f_transform_shifted, energy_percent)

        # Espectro filtrado para visualizar
        filtered_magnitude_spectrum = np.log1p(np.abs(filtered_transform))
        
        # Reconstrucción de la imagen desde la transformada filtrada
        reconstructed_img = np.abs(ifftn(filtered_transform))

        # Visualización de resultados
        plt.figure(figsize=(16, 4))
        plt.subplot(1, 4, 1)
        plt.imshow(img, cmap='gray')
        plt.title('Imagen Original')
        plt.axis('off')
        
        plt.subplot(1, 4, 2)
        plt.imshow(magnitude_spectrum, cmap='inferno')
        plt.title('Transformada de Fourier 2D')
        plt.axis('off')
        
        plt.subplot(1, 4, 3)
        plt.imshow(filtered_magnitude_spectrum, cmap='inferno')
        plt.title(f'Espectro Filtrado ({energy_percent}% energía)' if not use_ring_filter else 'Filtro de Anillo')
        plt.axis('off')
        
        plt.subplot(1, 4, 4)
        plt.imshow(reconstructed_img, cmap='gray')
        plt.title('Imagen Reconstruida')
        plt.axis('off')
        
        plt.show()

# Ejecutar con filtro de energía
process_images_in_folder("C:\\Users\\danwo\\Desktop\\ZS0001_TI_25XW_Rhodopsin_Au", energy_percent=20)

#Ejecutar con filtro de anillo 
#process_images_in_folder("C:\\Users\\danwo\\Desktop\\ZS0001_TI_25XW_Rhodopsin_Au", use_ring_filter=True, low_radius=0, high_radius=60)


In [ ]:
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt
from scipy.fftpack import fftn, fftshift, ifftn
from scipy.ndimage import rotate

# Función para cargar una imagen en escala de grises
def load_image(image_path):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        raise ValueError(f"No se pudo cargar la imagen: {image_path}")
    return img

# Función para calcular la Transformada de Fourier 2D
def compute_fft_2d(img):
    f_transform = fftn(img)  
    f_transform_shifted = fftshift(f_transform)  
    magnitude_spectrum = np.log1p(np.abs(f_transform_shifted))  
    return f_transform_shifted, magnitude_spectrum

# Filtro por porcentaje de energía
def filter_by_energy(f_transform, energy_percent=20):
    magnitude = np.abs(f_transform)
    total_energy = np.sum(magnitude)  # Energía total de la imagen en la frecuencia
    sorted_magnitude = np.sort(magnitude.ravel())[::-1]  # Ordenamos de mayor a menor
    cumulative_energy = np.cumsum(sorted_magnitude)  # Suma acumulativa de energía
    
    # Encontramos el umbral donde alcanzamos el porcentaje de energía deseado
    threshold_index = np.searchsorted(cumulative_energy, total_energy * (energy_percent / 100))
    threshold_value = sorted_magnitude[threshold_index]

    # Creamos la máscara conservando solo los valores con suficiente energía
    mask = magnitude >= threshold_value
    filtered_transform = f_transform * mask  # Aplicamos el filtro
    
    print(f"Umbral de energía aplicado: {threshold_value:.4f} (Conserva {energy_percent}% de la energía)")
    return filtered_transform, mask

# Calcular la dirección principal del espectro filtrado
def calculate_main_direction(mask):
    h, w = mask.shape
    cy, cx = h // 2, w // 2  # Centro del espectro

    # Coordenadas de los puntos activados en la máscara
    y, x = np.nonzero(mask)
    y = y - cy
    x = x - cx

    # Calcular el ángulo principal usando el momento de inercia
    cov_matrix = np.cov(x, y)
    eigenvalues, eigenvectors = np.linalg.eig(cov_matrix)
    main_direction = np.arctan2(eigenvectors[1, 0], eigenvectors[0, 0])  # Ángulo en radianes
    return np.degrees(main_direction)  # Convertir a grados

# Filtro de dos líneas delgadas rotadas
def filter_two_lines_rotated(f_transform, angle):
    h, w = f_transform.shape
    cy, cx = h // 2, w // 2  # Centro del espectro

    # Crear una máscara con dos líneas delgadas
    mask = np.zeros((h, w), dtype=np.uint8)

    # Línea horizontal
    mask[cy - 1:cy + 2, :] = 1  # Grosor de 3 píxeles

    # Línea vertical
    mask[:, cx - 1:cx + 2] = 1  # Grosor de 3 píxeles

    # Rotar la máscara
    rotated_mask = rotate(mask, angle, reshape=False, order=1)
    filtered_transform = f_transform * rotated_mask
    return filtered_transform, rotated_mask

# Función principal para procesar todas las imágenes en una carpeta
def process_images_with_rotated_filter(folder_path):
    image_files = sorted([f for f in os.listdir(folder_path) if f.endswith(('.png', '.jpg', '.tif'))])
    if not image_files:
        raise ValueError("No se encontraron imágenes en la carpeta")

    for image_file in image_files:
        image_path = os.path.join(folder_path, image_file)
        img = load_image(image_path)  # Cargar la imagen
        f_transform_shifted, magnitude_spectrum = compute_fft_2d(img)  # Transformada de Fourier 2D
        
        # Aplicar el filtro por energía al 20%
        filtered_transform_energy, mask_energy = filter_by_energy(f_transform_shifted, energy_percent=20)

        # Calcular la dirección principal del espectro filtrado
        main_direction = calculate_main_direction(mask_energy)
        print(f"Dirección principal calculada: {main_direction:.2f} grados")

        # Aplicar el filtro de dos líneas rotadas
        filtered_transform_lines, rotated_mask = filter_two_lines_rotated(f_transform_shifted, main_direction)

        # Espectros filtrados para visualizar
        filtered_magnitude_spectrum_energy = np.log1p(np.abs(filtered_transform_energy))
        filtered_magnitude_spectrum_lines = np.log1p(np.abs(filtered_transform_lines))
        
        # Reconstrucción de la imagen desde la transformada filtrada
        reconstructed_img_energy = np.abs(ifftn(filtered_transform_energy))
        reconstructed_img_lines = np.abs(ifftn(filtered_transform_lines))

        # Visualización de resultados
        plt.figure(figsize=(20, 8))
        plt.subplot(2, 4, 1)
        plt.imshow(img, cmap='gray')
        plt.title('Imagen Original')
        plt.axis('off')
        
        plt.subplot(2, 4, 2)
        plt.imshow(magnitude_spectrum, cmap='inferno')
        plt.title('Transformada de Fourier 2D')
        plt.axis('off')
        
        plt.subplot(2, 4, 3)
        plt.imshow(mask_energy, cmap='gray')
        plt.title('Máscara del Filtro (20% Energía)')
        plt.axis('off')
        
        plt.subplot(2, 4, 4)
        plt.imshow(filtered_magnitude_spectrum_energy, cmap='inferno')
        plt.title('Espectro Filtrado (20% Energía)')
        plt.axis('off')
        
        plt.subplot(2, 4, 5)
        plt.imshow(rotated_mask, cmap='gray')
        plt.title(f'Máscara Rotada ({main_direction:.2f}°)')
        plt.axis('off')
        
        plt.subplot(2, 4, 6)
        plt.imshow(filtered_magnitude_spectrum_lines, cmap='inferno')
        plt.title('Espectro Filtrado (Líneas Rotadas)')
        plt.axis('off')
        
        plt.subplot(2, 4, 7)
        plt.imshow(reconstructed_img_energy, cmap='gray')
        plt.title('Imagen Reconstruida (20% Energía)')
        plt.axis('off')
        
        plt.subplot(2, 4, 8)
        plt.imshow(reconstructed_img_lines, cmap='gray')
        plt.title('Imagen Reconstruida (Líneas Rotadas)')
        plt.axis('off')
        
        plt.tight_layout()
        plt.show()

# Ejecutar el procesamiento con filtro rotado
process_images_with_rotated_filter("C:\\Users\\danwo\\Desktop\\ZS0001_TI_25XW_Rhodopsin_Au")

In [ ]:
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt
from scipy.fftpack import fftn, fftshift, ifftn
from scipy.ndimage import rotate

def load_image(image_path):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        raise ValueError(f"No se pudo cargar la imagen: {image_path}")
    return img

def compute_fft_2d(img):
    f_transform = fftn(img)
    f_transform_shifted = fftshift(f_transform)
    magnitude_spectrum = np.log1p(np.abs(f_transform_shifted))
    return f_transform_shifted, magnitude_spectrum

def filter_by_energy(f_transform, energy_percent=20):
    magnitude = np.abs(f_transform)
    total_energy = np.sum(magnitude)
    sorted_magnitude = np.sort(magnitude.ravel())[::-1]
    cumulative_energy = np.cumsum(sorted_magnitude)
    threshold_index = np.searchsorted(cumulative_energy, total_energy * (energy_percent / 100))
    threshold_value = sorted_magnitude[threshold_index]
    mask = magnitude >= threshold_value
    filtered_transform = f_transform * mask
    return filtered_transform, mask

def detect_main_directions(mask):
    mask_uint8 = (mask * 255).astype(np.uint8)
    edges = cv2.Canny(mask_uint8, 50, 150)
    lines = cv2.HoughLines(edges, 1, np.pi / 180, threshold=50)
    if lines is not None:
        angles = [theta for _, theta in lines[:, 0, :]]
        main_angle = np.median(angles)  # Tomamos la mediana como el ángulo principal
        return np.degrees(main_angle) - 90  # Ajuste a la representación esperada
    return 0

def filter_two_lines_rotated(f_transform, angle):
    h, w = f_transform.shape
    cy, cx = h // 2, w // 2
    mask = np.zeros((h, w), dtype=np.uint8)
    cv2.line(mask, (0, cy), (w, cy), 1, 1)
    cv2.line(mask, (cx, 0), (cx, h), 1, 1)
    rotated_mask = rotate(mask, angle, reshape=False, order=1)
    filtered_transform = f_transform * rotated_mask
    return filtered_transform, rotated_mask

def process_images_with_rotated_filter(folder_path):
    image_files = sorted([f for f in os.listdir(folder_path) if f.endswith(('.png', '.jpg', '.tif'))])
    for image_file in image_files:
        image_path = os.path.join(folder_path, image_file)
        img = load_image(image_path)
        f_transform_shifted, magnitude_spectrum = compute_fft_2d(img)
        filtered_transform_energy, mask_energy = filter_by_energy(f_transform_shifted, energy_percent=20)
        main_direction = detect_main_directions(mask_energy)
        filtered_transform_lines, rotated_mask = filter_two_lines_rotated(f_transform_shifted, main_direction)
        reconstructed_img_energy = np.abs(ifftn(filtered_transform_energy))
        reconstructed_img_lines = np.abs(ifftn(filtered_transform_lines))
        plt.figure(figsize=(20, 8))
        plt.subplot(2, 4, 1)
        plt.imshow(img, cmap='gray')
        plt.title('Imagen Original')
        plt.axis('off')
        plt.subplot(2, 4, 2)
        plt.imshow(magnitude_spectrum, cmap='inferno')
        plt.title('Transformada de Fourier 2D')
        plt.axis('off')
        plt.subplot(2, 4, 3)
        plt.imshow(mask_energy, cmap='gray')
        plt.title('Máscara del Filtro (20% Energía)')
        plt.axis('off')
        plt.subplot(2, 4, 4)
        plt.imshow(np.log1p(np.abs(filtered_transform_energy)), cmap='inferno')
        plt.title('Espectro Filtrado (20% Energía)')
        plt.axis('off')
        plt.subplot(2, 4, 5)
        plt.imshow(rotated_mask, cmap='gray')
        plt.title(f'Máscara Rotada ({main_direction:.2f}°)')
        plt.axis('off')
        plt.subplot(2, 4, 6)
        plt.imshow(np.log1p(np.abs(filtered_transform_lines)), cmap='inferno')
        plt.title('Espectro Filtrado (Líneas Rotadas)')
        plt.axis('off')
        plt.subplot(2, 4, 7)
        plt.imshow(reconstructed_img_energy, cmap='gray')
        plt.title('Imagen Reconstruida (20% Energía)')
        plt.axis('off')
        plt.subplot(2, 4, 8)
        plt.imshow(reconstructed_img_lines, cmap='gray')
        plt.title('Imagen Reconstruida (Líneas Rotadas)')
        plt.axis('off')
        plt.tight_layout()
        plt.show()

# Ejecutar el procesamiento con filtro rotado
directory = "C:\\Users\\danwo\\Desktop\\ZS0001_TI_25XW_Rhodopsin_Au"
process_images_with_rotated_filter(directory)


In [ ]:
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt
from scipy.fftpack import fftn, fftshift, ifftn
from scipy.ndimage import rotate

def load_image(image_path):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        raise ValueError(f"No se pudo cargar la imagen: {image_path}")
    return img

def compute_fft_2d(img):
    f_transform = fftn(img)
    f_transform_shifted = fftshift(f_transform)
    magnitude_spectrum = np.log1p(np.abs(f_transform_shifted))
    return f_transform_shifted, magnitude_spectrum

def filter_by_energy(f_transform, energy_percent=20):
    magnitude = np.abs(f_transform)
    total_energy = np.sum(magnitude)
    sorted_magnitude = np.sort(magnitude.ravel())[::-1]
    cumulative_energy = np.cumsum(sorted_magnitude)
    threshold_index = np.searchsorted(cumulative_energy, total_energy * (energy_percent / 100))
    threshold_value = sorted_magnitude[threshold_index]
    mask = magnitude >= threshold_value
    filtered_transform = f_transform * mask
    return filtered_transform, mask

def detect_main_directions(mask):
    mask_uint8 = (mask * 255).astype(np.uint8)
    edges = cv2.Canny(mask_uint8, 50, 150)
    lines = cv2.HoughLines(edges, 1, np.pi / 180, threshold=50)
    if lines is not None:
        angles = [theta for _, theta in lines[:, 0, :]]
        main_angle = np.median(angles)  # Tomamos la mediana como el ángulo principal
        return np.degrees(main_angle) - 90  # Ajuste a la representación esperada
    return 0

def filter_two_lines_rotated(f_transform, angle):
    h, w = f_transform.shape
    cy, cx = h // 2, w // 2
    mask = np.zeros((h, w), dtype=np.uint8)
    cv2.line(mask, (0, cy), (w, cy), 1, 3)  # Línea más gruesa
    cv2.line(mask, (cx, 0), (cx, h), 1, 3)  # Línea más gruesa
    rotated_mask = rotate(mask, angle, reshape=False, order=1)
    filtered_transform = f_transform * rotated_mask
    return filtered_transform, rotated_mask

def process_images_with_rotated_filter(folder_path):
    image_files = sorted([f for f in os.listdir(folder_path) if f.endswith(('.png', '.jpg', '.tif'))])
    for image_file in image_files:
        image_path = os.path.join(folder_path, image_file)
        img = load_image(image_path)
        f_transform_shifted, magnitude_spectrum = compute_fft_2d(img)
        filtered_transform_energy, mask_energy = filter_by_energy(f_transform_shifted, energy_percent=20)
        main_direction = detect_main_directions(mask_energy)
        filtered_transform_lines, rotated_mask = filter_two_lines_rotated(f_transform_shifted, main_direction)
        reconstructed_img_energy = np.abs(ifftn(filtered_transform_energy))
        reconstructed_img_lines = np.abs(ifftn(filtered_transform_lines))
        plt.figure(figsize=(20, 8))
        plt.subplot(2, 4, 1)
        plt.imshow(img, cmap='gray')
        plt.title('Imagen Original')
        plt.axis('off')
        plt.subplot(2, 4, 2)
        plt.imshow(magnitude_spectrum, cmap='inferno')
        plt.title('Transformada de Fourier 2D')
        plt.axis('off')
        plt.subplot(2, 4, 3)
        plt.imshow(mask_energy, cmap='gray')
        plt.title('Máscara del Filtro (20% Energía)')
        plt.axis('off')
        plt.subplot(2, 4, 4)
        plt.imshow(np.log1p(np.abs(filtered_transform_energy)), cmap='inferno')
        plt.title('Espectro Filtrado (20% Energía)')
        plt.axis('off')
        plt.subplot(2, 4, 5)
        plt.imshow(rotated_mask, cmap='gray')
        plt.title(f'Máscara Rotada ({main_direction:.2f}°)')
        plt.axis('off')
        plt.subplot(2, 4, 6)
        plt.imshow(np.log1p(np.abs(filtered_transform_lines)), cmap='inferno')
        plt.title('Espectro Filtrado (Líneas Rotadas)')
        plt.axis('off')
        plt.subplot(2, 4, 7)
        plt.imshow(reconstructed_img_energy, cmap='gray')
        plt.title('Imagen Reconstruida (20% Energía)')
        plt.axis('off')
        plt.subplot(2, 4, 8)
        plt.imshow(reconstructed_img_lines, cmap='gray')
        plt.title('Imagen Reconstruida (Líneas Rotadas)')
        plt.axis('off')
        plt.tight_layout()
        plt.show()

# Ejecutar el procesamiento con filtro rotado
directory = "C:\\Users\\danwo\\Desktop\\ZS0001_TI_25XW_Rhodopsin_Au"
process_images_with_rotated_filter(directory)


Ahora vamos hacer lo mismo pero con otro stack de imágenes para ver que pasa

In [ ]:
# Ejecutar el procesamiento con filtro rotado
#directory = "C:\\Users\\danwo\\Desktop\\GRADO_FÍSICA\\Máster\\Segundo_cuatri\\TFM\\Images_Oculartissue\\Images_Oculartissue\\01_Sclera\\01_wholeEyes\\20220518_Exps\\Sample2\\ZS0004_TI_25XW_Sample2_MB_1_2A1"
directory = "C:\\Users\\danwo\\Desktop\\ZS0004_TI_25XW_Sample2_MB_1_2A1"
process_images_with_rotated_filter(directory)

In [ ]:
# Ejecutar el procesamiento con filtro rotado
directory = "C:\\Users\\danwo\\Desktop\\ZS0018_TI_25XW_Strip_4A1"
process_images_with_rotated_filter(directory)

In [ ]:
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt
from scipy.fftpack import fftn, fftshift, ifftn
from scipy.ndimage import rotate
from scipy.ndimage import binary_opening

# Filtrar puntos dispersos en la máscara
def clean_mask(mask):
    cleaned_mask = binary_opening(mask, structure=np.ones((3, 3)))  # Filtro morfológico
    return cleaned_mask

# Función para cargar una imagen en escala de grises
def load_image(image_path):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        raise ValueError(f"No se pudo cargar la imagen: {image_path}")
    return img

# Función para calcular la Transformada de Fourier 2D
def compute_fft_2d(img):
    f_transform = fftn(img)  
    f_transform_shifted = fftshift(f_transform)  
    magnitude_spectrum = np.log1p(np.abs(f_transform_shifted))  
    return f_transform_shifted, magnitude_spectrum

# Filtro por porcentaje de energía
def filter_by_energy(f_transform, energy_percent=20):
    magnitude = np.abs(f_transform)
    total_energy = np.sum(magnitude)  # Energía total de la imagen en la frecuencia
    sorted_magnitude = np.sort(magnitude.ravel())[::-1]  # Ordenamos de mayor a menor
    cumulative_energy = np.cumsum(sorted_magnitude)  # Suma acumulativa de energía
    
    # Encontramos el umbral donde alcanzamos el porcentaje de energía deseado
    threshold_index = np.searchsorted(cumulative_energy, total_energy * (energy_percent / 100))
    threshold_value = sorted_magnitude[threshold_index]

    # Creamos la máscara conservando solo los valores con suficiente energía
    mask = magnitude >= threshold_value
    filtered_transform = f_transform * mask  # Aplicamos el filtro
    
    print(f"Umbral de energía aplicado: {threshold_value:.4f} (Conserva {energy_percent}% de la energía)")
    return filtered_transform, mask

# Calcular la dirección principal del espectro filtrado
def calculate_main_direction(mask, magnitude= None, method="covariance"):
    """
    Calcula la dirección principal del espectro filtrado.
    
    Parámetros:
        mask (ndarray): Máscara binaria del espectro.
        magnitude (ndarray, opcional): Magnitud del espectro para ponderar los puntos.
        method (str): Método para calcular la dirección ("covariance" o "least_squares").
    
    Retorna:
        float: Dirección principal en grados.
    """
    h, w = mask.shape
    cy, cx = h // 2, w // 2  # Centro del espectro

    # Filtrar puntos dispersos
    mask = clean_mask(mask)

    # Coordenadas de los puntos activados en la máscara
    y, x = np.nonzero(mask)
    y = y - cy
    x = x - cx

    if method == "covariance":
        # Ponderar por la magnitud si se proporciona
        if magnitude is not None:
            magnitude = np.abs(magnitude)  # Asegurarse de que no haya valores negativos
            weights = magnitude[mask]
            cov_matrix = np.cov(x, y, aweights=weights)
        else:
            cov_matrix = np.cov(x, y)
        
        # Calcular valores y vectores propios
        eigenvalues, eigenvectors = np.linalg.eig(cov_matrix)
        main_direction = np.arctan2(eigenvectors[1, 0], eigenvectors[0, 0])  # Ángulo en radianes

    elif method == "least_squares":
        # Ajustar una línea usando mínimos cuadrados
        A = np.vstack([x, np.ones(len(x))]).T
        m, _ = np.linalg.lstsq(A, y, rcond=None)[0]
        main_direction = np.arctan(m)  # Ángulo en radianes

    else:
        raise ValueError("Método no reconocido. Usa 'covariance' o 'least_squares'.")

    return np.degrees(main_direction)  # Convertir a grados

# Filtro de dos líneas delgadas rotadas
def filter_two_lines_rotated(f_transform, angle):
    h, w = f_transform.shape
    cy, cx = h // 2, w // 2  # Centro del espectro

    # Crear una máscara con dos líneas delgadas
    mask = np.zeros((h, w), dtype=np.uint8)

    # Línea horizontal
    mask[cy - 1:cy + 2, :] = 1  # Grosor de 3 píxeles

    # Línea vertical
    mask[:, cx - 1:cx + 2] = 1  # Grosor de 3 píxeles

    # Rotar la máscara
    rotated_mask = rotate(mask, angle, reshape=False, order=1)
    filtered_transform = f_transform * rotated_mask
    return filtered_transform, rotated_mask

# Función principal para procesar todas las imágenes en una carpeta
def process_images_with_rotated_filter(folder_path):
    image_files = sorted([f for f in os.listdir(folder_path) if f.endswith(('.png', '.jpg', '.tif'))])
    if not image_files:
        raise ValueError("No se encontraron imágenes en la carpeta")

    for image_file in image_files:
        image_path = os.path.join(folder_path, image_file)
        img = load_image(image_path)  # Cargar la imagen
        f_transform_shifted, magnitude_spectrum = compute_fft_2d(img)  # Transformada de Fourier 2D
        
        # Aplicar el filtro por energía al 20%
        filtered_transform_energy, mask_energy = filter_by_energy(f_transform_shifted, energy_percent=20)

        # Calcular la dirección principal del espectro filtrado
        main_direction = calculate_main_direction(mask_energy, magnitude=f_transform_shifted, method="least_squares")
        print(f"Dirección principal calculada: {main_direction:.2f} grados")

        # Aplicar el filtro de dos líneas rotadas
        filtered_transform_lines, rotated_mask = filter_two_lines_rotated(f_transform_shifted, main_direction)

        # Espectros filtrados para visualizar
        filtered_magnitude_spectrum_energy = np.log1p(np.abs(filtered_transform_energy))
        filtered_magnitude_spectrum_lines = np.log1p(np.abs(filtered_transform_lines))
        
        # Reconstrucción de la imagen desde la transformada filtrada
        reconstructed_img_energy = np.abs(ifftn(filtered_transform_energy))
        reconstructed_img_lines = np.abs(ifftn(filtered_transform_lines))

        # Visualización de resultados
        plt.figure(figsize=(20, 8))
        plt.subplot(2, 4, 1)
        plt.imshow(img, cmap='gray')
        plt.title('Imagen Original')
        plt.axis('off')
        
        plt.subplot(2, 4, 2)
        plt.imshow(magnitude_spectrum, cmap='inferno')
        plt.title('Transformada de Fourier 2D')
        plt.axis('off')
        
        plt.subplot(2, 4, 3)
        plt.imshow(mask_energy, cmap='gray')
        plt.title('Máscara del Filtro (20% Energía)')
        plt.axis('off')
        
        plt.subplot(2, 4, 4)
        plt.imshow(filtered_magnitude_spectrum_energy, cmap='inferno')
        plt.title('Espectro Filtrado (20% Energía)')
        plt.axis('off')
        
        plt.subplot(2, 4, 5)
        plt.imshow(rotated_mask, cmap='gray')
        plt.title(f'Máscara Rotada ({main_direction:.2f}°)')
        plt.axis('off')
        
        plt.subplot(2, 4, 6)
        plt.imshow(filtered_magnitude_spectrum_lines, cmap='inferno')
        plt.title('Espectro Filtrado (Líneas Rotadas)')
        plt.axis('off')
        
        plt.subplot(2, 4, 7)
        plt.imshow(reconstructed_img_energy, cmap='gray')
        plt.title('Imagen Reconstruida (20% Energía)')
        plt.axis('off')
        
        plt.subplot(2, 4, 8)
        plt.imshow(reconstructed_img_lines, cmap='gray')
        plt.title('Imagen Reconstruida (Líneas Rotadas)')
        plt.axis('off')
        
        plt.tight_layout()
        plt.show()

# Ejecutar el procesamiento con filtro rotado
process_images_with_rotated_filter("C:\\Users\\danwo\\Desktop\\ZS0001_TI_25XW_Rhodopsin_Au")